# Skyview masking

Create skyview mask from fisheye photo. The mask should be simplified, can be fuzzy. Workflow:

1. Open images
2. Find tresholds for sky, use blue, possible also red and green band.
3. Create fuzzy mask
4. Create hard mask
5. Fill smaller areas

In [ ]:
# Libraries
from skimage import io
from skimage import filters
from skimage import morphology
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os

In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)

## Get images

In [ ]:
"./SkyView/originali/"
# Cropped photo location
photo_folder = "./SkyView/"

In [ ]:
# Read all photos
photos = glob.glob(photo_folder + '2019_*.jpg')
photos

In [ ]:
# Plot image, mask
def treshold_plot (band, mask, title = ""):
    fig, axes = plt.subplots(ncols=2, figsize=(15, 7))
    fig.suptitle(title)
    
    ax = axes.ravel()
    ax[0] = plt.subplot(1, 2, 1)
    ax[1] = plt.subplot(1, 2, 2, sharex=ax[0], sharey=ax[0])

    ax[0].imshow(band, cmap=plt.cm.gray)
    ax[0].set_title('Original')
    ax[0].axis('off')

    ax[1].imshow(mask, cmap=plt.cm.gray)
    ax[1].set_title('Thresholded')
    ax[1].axis('off')

    plt.show()

## Create masks

All bands are tresholded and the mask is combined. We are using Otsu filter tresholds. At the end we create a hard mask.

In [ ]:
photo = photos[0]
for photo in photos:
    image = io.imread(photo)
    name = os.path.basename(photo)
    print("Masking", name)
    # Find tresholds
    tresh = np.empty(3, dtype=int)
    for img_b in range(3):
        img = image[:,:,img_b]
        tresh[img_b] = filters.threshold_otsu(img)
    print("Treshold", tresh)
    # Create fuzzy mask
    mask = np.copy(image[:,:,0]) * 0
    for i in range(3):
        mask_add = image[:,:,i] >= tresh[i]
        mask[:,:] = mask + mask_add
    # Hard mask, at least in one band
    mask_h = mask > 1
    # Save mask
    mask_h_im = Image.fromarray(mask_h * 255)
    mask_h_im = mask_h_im.convert("L")
    photo_out = photo.rsplit('.', 1)[0] + "_mask"+ ".png"
    mask_h_im.save(photo_out)
    print("Saved.")